In [21]:
# Import of librairies
import tensorflow as tf
import mysql.connector as mariadb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tqdm import tqdm
from joblib import load
from timeloop import Timeloop
from datetime import timedelta
import time 

class sql_query:
    def __init__(self, credentials_path):
        self.db_credentials = pd.read_csv(credentials_path, index_col="Field")
      
    
    def __call__(self, query):
        
        mariadb_connection = mariadb.connect(
            user=self.db_credentials.loc["user"][0],
            password=self.db_credentials.loc["password"][0],
            host=self.db_credentials.loc["host"][0],
            port=3306,
            db = "db_velib")
        
        self.cursor = mariadb_connection.cursor()
        cursor = self.cursor
        cursor.execute("SET  time_zone = 'Europe/Paris'")
        cursor.execute(query)
        field_names = [i[0] for i in cursor.description]
        df = pd.DataFrame(cursor, columns=field_names)
        return df
    
# Transforming the input data in the proper format 


def measure_rmse(actual, predicted):
    return math.sqrt(mean_squared_error(actual, predicted))

def list_stations():
    request = sql_query("../../aws_mariadb_crendentials.csv")
    query = """
    SELECT DISTINCT station_id FROM velib_realtime
    """
    df= request(query)
    # Removing bad values
    df= df.drop(0)
    df = df.drop(1391)
    list_of_stations = list(df.station_id)
    return list_of_stations

def loading_models_unique(station_id, day_of_testing):

    try:
        LSTM_A = tf.keras.models.load_model('../4. Models/Tensorflow Univariate - {} - {} - LSTM_A.h5'.format(day_of_testing, station_id))
        LSTM_B = tf.keras.models.load_model('../4. Models/Tensorflow Univariate - {} - {} - LSTM_B.h5'.format(day_of_testing, station_id))
        std = load('../4. Models/Tensorflow Univariate - {} - {} - std.joblib'.format(day_of_testing, station_id))
        return LSTM_A, LSTM_B, std
    
    except:
        print('impossible to load ', list_of_stations[i])


def create_result_df():
    # Extracting base for prediction 

    request = sql_query("../../aws_mariadb_crendentials.csv")

    query = """
    SELECT station_id, date_of_update, nb_total_free_bikes FROM db_velib.velib_realtime
    WHERE date_of_update >= DATE_SUB(NOW(), INTERVAL 200 Minute) AND MINUTE(date_of_update)%5 = 0
    ORDER BY station_id, date_of_update ASC;
    """
    df= request(query)
    df.index = df.date_of_update
    df = df[['station_id','nb_total_free_bikes']]
    df = df.pivot_table(df, index= 'station_id', columns=df.index)

    # Creating dataframe for proper predction

    df_prediction = pd.DataFrame(index=df.index, columns=['last_observations','model_A', 'model_B', 'date_of_prediction'])

    for i in df_prediction.index:
        df_prediction["last_observations"].loc[i] = np.array(df.loc[i])
    
    df_prediction['date_of_prediction'] = str(pd.Timestamp.now())[:16]
    return df_prediction


def predict_iteration_unique(list_of_stations, df_prediction, LSTM_A, LSTM_B, std):
    # Request for each minutes
    for station_id in tqdm(list_of_stations):
        try:
            input_data = std.transform(df_prediction[df_prediction.index == station_id]["last_observations"].iloc[0].reshape(-1, 1))[-36:]
            df_prediction.loc[station_id]['model_A'] = std.inverse_transform(LSTM_A.predict(input_data.reshape(1,past_history,1))[0])
            df_prediction.loc[station_id]['model_B'] = std.inverse_transform(LSTM_B.predict(input_data.reshape(1,past_history,1))[0])
            df_prediction
        except:
            print('error on ', station_id)
# Initialisation

# Variables
day_of_testing = '2020-05-19'

#

In [22]:
station_id = 'global'
past_history = 36
future_target = 6
list_of_stations = list_stations()[:20]

# Loading the models only once
LSTM_A, LSTM_B, std = loading_models_unique(station_id, day_of_testing)
df_prediction = create_result_df()



In [31]:
df_prediction.model_A[1]

KeyError: 1

In [23]:
station_id = list_of_stations[0]

In [24]:
station_id = list_of_stations[0]

input_data = std.transform(df_prediction[df_prediction.index == station_id]["last_observations"].iloc[0].reshape(-1, 1))[-36:]
df_prediction.loc[station_id]['model_A'] = std.inverse_transform(LSTM_A.predict(input_data.reshape(1,past_history,1))[0])
df_prediction.loc[station_id]['model_B'] = std.inverse_transform(LSTM_B.predict(input_data.reshape(1,past_history,1))[0])
df_prediction

,last_observations,model_A,model_B,date_of_prediction
station_id,,,,
1001,"[14, 16, 14, 15, 17, 15, 14, 15, 12, 14, 15, 1...","[11.114539, 11.149158, 11.144817, 11.129019, 1...","[11.314236, 11.257081, 11.236649, 11.115834, 1...",2020-06-09 22:03
1002,"[20, 18, 19, 21, 21, 17, 17, 19, 20, 22, 21, 2...",NaN,NaN,2020-06-09 22:03
1003,"[37, 38, 37, 37, 38, 38, 40, 41, 40, 41, 41, 4...",NaN,NaN,2020-06-09 22:03
1006,"[29, 30, 30, 29, 32, 30, 30, 29, 28, 26, 29, 2...",NaN,NaN,2020-06-09 22:03
1007,"[13, 11, 11, 11, 13, 14, 14, 14, 14, 14, 15, 1...",NaN,NaN,2020-06-09 22:03
...,...,...,...,...
48012,"[2, 2, 4, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 1, ...",NaN,NaN,2020-06-09 22:03
51001,"[20, 19, 19, 19, 19, 19, 19, 18, 17, 16, 16, 1...",NaN,NaN,2020-06-09 22:03
51002,"[2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 4, 4, 5, 5, ...",NaN,NaN,2020-06-09 22:03


In [ ]:
predict_iteration_unique(list_of_stations, df_prediction, LSTM_A, LSTM_B, std)

In [6]:

df_prediction.to_csv('../7. Predictions/5min_predictions_global.csv')


 37%|███▋      | 520/1390 [00:00<00:00, 1566.33it/s]

error on  1001
error on  1002
error on  1003
error on  1006
error on  1007
error on  1008
error on  1012
error on  1013
error on  1014
error on  1015
error on  1016
error on  1017
error on  1018
error on  1020
error on  1021
error on  1022
error on  1023
error on  1024
error on  1025
error on  1026
error on  1102
error on  1116
error on  1117
error on  1119
error on  1120
error on  1122
error on  2001
error on  2002
error on  2003
error on  2004
error on  2005
error on  2006
error on  2008
error on  2009
error on  2010
error on  2012
error on  2013
error on  2014
error on  2015
error on  2016
error on  2017
error on  2020
error on  2021
error on  2022
error on  2101
error on  2102
error on  2108
error on  2109
error on  2110
error on  2111
error on  2112
error on  3001
error on  3002
error on  3003
error on  3004
error on  3005
error on  3006
error on  3007
error on  3008
error on  3009
error on  3010
error on  3011
error on  3012
error on  3101
error on  3102
error on  4001
error on  

 74%|███████▍  | 1031/1390 [00:00<00:00, 1641.91it/s]

error on  13101
error on  13102
error on  13103
error on  13104
error on  13106
error on  13109
error on  13110
error on  13113
error on  13114
error on  13116
error on  13117
error on  13118
error on  13120
error on  13121
error on  13122
error on  13123
error on  13125
error on  13126
error on  13128
error on  13515
error on  13516
error on  14001
error on  14002
error on  14003
error on  14004
error on  14005
error on  14006
error on  14007
error on  14008
error on  14009
error on  14010
error on  14011
error on  14012
error on  14013
error on  14014
error on  14016
error on  14018
error on  14019
error on  14020
error on  14021
error on  14022
error on  14023
error on  14025
error on  14026
error on  14027
error on  14028
error on  14030
error on  14031
error on  14032
error on  14034
error on  14035
error on  14036
error on  14101
error on  14103
error on  14104
error on  14106
error on  14108
error on  14110
error on  14111
error on  14112
error on  14113
error on  14116
error on

 86%|████████▌ | 1192/1390 [00:00<00:00, 1446.89it/s]

error on  21030
error on  21031
error on  21032
error on  21104
error on  21105
error on  21107
error on  21108
error on  21109
error on  21110
error on  21111
error on  21112
error on  21114
error on  21115
error on  21118
error on  21119
error on  21203
error on  21204
error on  21205
error on  21207
error on  21209
error on  21212
error on  21213
error on  21215
error on  21216
error on  21217
error on  21219
error on  21301
error on  21302
error on  21303
error on  21305
error on  21306
error on  21307
error on  21308
error on  21309
error on  21310
error on  21311
error on  21312
error on  21313
error on  21314
error on  21315
error on  21316
error on  21319
error on  21401
error on  21402
error on  21403
error on  21501
error on  21502
error on  21505
error on  21506
error on  21507
error on  21701
error on  21702
error on  21703
error on  21704
error on  21706
error on  21707
error on  21708
error on  21951
error on  21952
error on  21953
error on  21954
error on  22001
error on

100%|██████████| 1390/1390 [00:00<00:00, 1585.87it/s]


error on  45002
error on  45003
error on  45004
error on  46001
error on  46002
error on  46003
error on  47001
error on  47002
error on  47003
error on  47004
error on  47005
error on  47006
error on  47007
error on  48002
error on  48003
error on  48004
error on  48005
error on  48006
error on  48007
error on  48008
error on  48009
error on  48010
error on  48012
error on  51001
error on  51002
error on  51003
